# Dynamic Query Expansion

This is an implementation of the Dynamic Query Expansion Algorithm for Twitter Data as explained in 
"Unsupervised Spatial Event Detection in Targeted Domains with Applications to Civil Unrest Modeling" by Liang Zhao et al.

The implementation is done in python 2.7.6 and it's dependencies are libraries json,nltk,numpy and scipy.

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import json
import io
import ast
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
import numpy
from scipy.sparse import lil_matrix

Loading Data from a precollected collection of tweets in a json File which is stored in a unicode format

In [2]:
filename='raw_tweets2.json'
data_json = io.open(filename, mode='r', encoding='utf-8').read() #reads in the JSON file
data_python = json.loads(data_json)

In [3]:
#dictionary to capture the frequency of every tweet feature like token,hashtag,url etc.. 
#will be used to find IDF for said features
features_freq=defaultdict(float)

#NLTK tokenizer for tweets which strips handles from the tweet text and reduces repetition of letters
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

Collecting frequencies of features for IDF identity matrix(Df)

In [4]:
count_tweets=0 #Count for number of tweets in Colombia
for line in data_python:
        d = ast.literal_eval(line)
        if d['location']=='CO': #Since the bounding box contains more countries, apart from Colombia
            count_tweets+=1 
            features_freq[d['name']]+=1.0 #adding screen name of tweet sender
            for r in d['hashtags']:
                 features_freq[r['text']]+=1.0 #adding hashtag's frequencies
            for r1 in d['urls']:
                features_freq[r1['expanded_url']]+=1.0 #Adding URL frequencies
            text_tok=unicode(d['text']).lower()
            list_tok=tknzr.tokenize(text_tok)
            for tok in list_tok:
                if tok.isalpha():
                    features_freq[tok]+=1.0

Making a list of unique features to use for populatin adjacency matrix

In [5]:
features_list=[]
for f in features_freq:
    features_list.append(f)

Adding each tweet to a list of tweets and assigning each tweet with corresponding features in a adjacency matrix.
The adjacency matrix is a sparse matrix from scipy(lil_matrix) to account for large feature list and tweets.

In [6]:
tweet_list=[] #List of Tweet IDs 
tweet_dict={} #Dictionary of tweet ID corrsponding to the text for final result

adj_mat1=lil_matrix((len(features_list),count_tweets),dtype=numpy.float) #Adjaceny matrix for Tweet and FeatureList

count=-1
for line1 in data_python:
    d1 = ast.literal_eval(line1)
    if d1['location']=='CO':
        count+=1 
        tweet_list.append(d1['id'])
        tweet_dict[d1['id']]=d1['text']
        for r in d1['hashtags']:
            adj_mat1[features_list.index(r['text']),count]=1
        for r1 in d1['urls']:
            adj_mat1[features_list.index(r1['expanded_url']),count]=1
        adj_mat1[features_list.index(d1['name']),count]=1
        text_tok1=unicode(d1['text']).lower()
        list_tok1=tknzr.tokenize(text_tok1)
        for tok1 in list_tok1:
            if tok1.isalpha():
                 adj_mat1[features_list.index(tok1),count]=1
print 'Tweet and Feature Matrix'
print adj_mat1

Tweet and Feature Matrix
  (0, 4220)	1.0
  (1, 4752)	1.0
  (2, 2339)	1.0
  (3, 4335)	1.0
  (4, 3673)	1.0
  (5, 1824)	1.0
  (6, 2420)	1.0
  (6, 3727)	1.0
  (7, 2460)	1.0
  (8, 4612)	1.0
  (9, 3588)	1.0
  (10, 4400)	1.0
  (11, 4222)	1.0
  (12, 2409)	1.0
  (13, 82)	1.0
  (13, 101)	1.0
  (13, 106)	1.0
  (13, 233)	1.0
  (13, 282)	1.0
  (13, 286)	1.0
  (13, 311)	1.0
  (13, 516)	1.0
  (13, 675)	1.0
  (13, 892)	1.0
  (13, 1192)	1.0
  (13, 1564)	1.0
  (13, 1807)	1.0
  (13, 1981)	1.0
  (13, 2264)	1.0
  (13, 2503)	1.0
  (13, 2771)	1.0
  (13, 2933)	1.0
  (13, 3212)	1.0
  (13, 3289)	1.0
  (13, 3428)	1.0
  (13, 3526)	1.0
  (13, 3777)	1.0
  (13, 3804)	1.0
  (13, 4134)	1.0
  (13, 4708)	1.0
  (13, 4766)	1.0
  (14, 833)	1.0
  (15, 4085)	1.0
  (16, 4108)	1.0
  (17, 3246)	1.0
  (18, 44)	1.0
  (18, 91)	1.0
  (18, 260)	1.0
  (18, 261)	1.0
  (18, 335)	1.0
  (18, 365)	1.0
  (18, 973)	1.0
  (18, 1648)	1.0
  (18, 1672)	1.0
  (18, 1678)	1.0
  (18, 1861)	1.0
  (18, 1875)	1.0
  (18, 2317)	1.0
  (18, 3233)	1.0
  (1

Populating a tweet-tweet adjacency matrix for modelling a reply relationship  

In [7]:
adj_mat2=lil_matrix((count_tweets,count_tweets),dtype=numpy.float)
for tw in data_python:
    t1=ast.literal_eval((tw))
    if t1['location']=='CO':
        rep_id=t1['reply']
        if rep_id is not None:
            if unicode(rep_id) in tweet_list:
                ind1=tweet_list.index(t1['id'])
                ind2=tweet_list.index(unicode(rep_id))
                adj_mat2[ind1,ind2]=1
                adj_mat2[ind2,ind1]=1
print 'Adjacency Matrix2:'
print adj_mat2

Adjacency Matrix2:
  (31, 2257)	1.0
  (58, 2271)	1.0
  (175, 276)	1.0
  (246, 1652)	1.0
  (276, 175)	1.0
  (332, 1008)	1.0
  (384, 387)	1.0
  (387, 384)	1.0
  (402, 760)	1.0
  (411, 467)	1.0
  (467, 411)	1.0
  (528, 558)	1.0
  (558, 528)	1.0
  (564, 2656)	1.0
  (586, 1296)	1.0
  (631, 796)	1.0
  (688, 799)	1.0
  (704, 718)	1.0
  (704, 856)	1.0
  (718, 704)	1.0
  (760, 402)	1.0
  (796, 631)	1.0
  (799, 688)	1.0
  (856, 704)	1.0
  (880, 1370)	1.0
  (1008, 332)	1.0
  (1088, 1105)	1.0
  (1105, 1088)	1.0
  (1118, 1167)	1.0
  (1118, 1197)	1.0
  (1167, 1118)	1.0
  (1197, 1118)	1.0
  (1296, 586)	1.0
  (1304, 1341)	1.0
  (1318, 1538)	1.0
  (1341, 1304)	1.0
  (1350, 1982)	1.0
  (1370, 880)	1.0
  (1385, 1410)	1.0
  (1410, 1385)	1.0
  (1538, 1318)	1.0
  (1573, 1826)	1.0
  (1576, 1577)	1.0
  (1577, 1576)	1.0
  (1581, 1675)	1.0
  (1652, 246)	1.0
  (1675, 1581)	1.0
  (1700, 2731)	1.0
  (1741, 2740)	1.0
  (1763, 3034)	1.0
  (1826, 1573)	1.0
  (1865, 1917)	1.0
  (1917, 1865)	1.0
  (1917, 1965)	1.0
  (1

Converting Feature frequencies to a Diagonal Matrix(Df)

In [8]:
idf_arr=numpy.zeros(len(features_list),dtype=numpy.float)
for i in range(0,len(features_list)):
    idf_arr[i]=1.0/features_freq[features_list[i]]
Df=lil_matrix((len(features_list),len(features_list)),dtype=numpy.float)
Df.setdiag(idf_arr)
print Df

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	0.5
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	0.037037037037
  (14, 14)	1.0
  (15, 15)	1.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	0.0434782608696
  (19, 19)	0.04
  (20, 20)	1.0
  (21, 21)	0.0588235294118
  (22, 22)	0.5
  (23, 23)	0.5
  (24, 24)	0.0833333333333
  (25, 25)	1.0
  (26, 26)	0.0769230769231
  (27, 27)	0.5
  (28, 28)	1.0
  (29, 29)	0.5
  (30, 30)	1.0
  (31, 31)	1.0
  (32, 32)	0.5
  (33, 33)	1.0
  (34, 34)	1.0
  (35, 35)	0.5
  (36, 36)	0.25
  (37, 37)	1.0
  (38, 38)	1.0
  (39, 39)	1.0
  (40, 40)	0.1
  (41, 41)	1.0
  (42, 42)	1.0
  (43, 43)	1.0
  (44, 44)	0.111111111111
  (45, 45)	1.0
  (46, 46)	1.0
  (47, 47)	0.5
  (48, 48)	0.5
  (49, 49)	0.25
  (50, 50)	1.0
  (51, 51)	1.0
  (52, 52)	1.0
  (53, 53)	1.0
  (54, 54)	1.0
  (55, 55)	0.111111111111
  (56, 56)	0.0357142857143
  (57, 57)	0.333333333333
  (58, 58)	1.0
  (59, 59)	1.0
  (60, 60)	1.

In [9]:
wt_tweets=numpy.zeros(count_tweets,dtype=numpy.float) #1D Array for tweet weights 
wt_features=numpy.zeros(len(features_list),dtype=numpy.float) #1D array for feature weights

Populating set of Tweets in seed-query. That is every tweet which contains tweets

In [10]:
seed_query=[u'violación',u'UNICEF',u'abuso',u'mujer'] #Seed Query terms

Tr=set()
for i in range(0,len(tweet_list)):
    for j in range(0,len(features_list)):
        if adj_mat1[j,i]==1:
            for sed in seed_query:
                #print sed
                #print features_list[j]
                if sed==features_list[j]:
                    Tr.add(i)
                    wt_tweets[i]=1.0

Printing Initial Set of Seed Queries

In [12]:
for t in Tr:
    print tweet_dict[tweet_list[t]]

Como es posible que si vas con tu mujer, en un SITP con un solo puesto libre, la hagas sentar en el piso para que tu te sientes en la silla!
Hoy NO es el día de las velitas. Es el día de la Virgen Inmaculada, única mujer que ha concebido sin pasar por el p… https://t.co/aPg3tCeqk7
Usted puede ser la niña mas linda, la mujer mas hermosa, pero si usted es o tiene delirios de ñera, no se me puede hacer atractiva
Ella Mi Reina Mi Gran Mujer La Dueña De Mi Mundo Este Amor Qué No… https://t.co/bFUlAbEald
@CaracolRadio DESPUES DE ESCUCHAR  ESTA ENTREVISTA EN CARACOL RADIO  ,   CORROBORO  QUE @SILVIACORZO  ES UNA MUJER  "EXTRAORDINARIA "
Me gusta la mujer que es frentera y no disimula 🎤
"El reggaeton denigra a la mujer"
- una morronga tercermundista que es moza o le acepta la moza al marido 

Lol la ignorancia😩
En San Cristóbal una mujer entrega en adopción a perros y gatos porque tiene que desalojar su casa. Informes en este teléfono 3102346729.
@AngieMmp buena decisión sabia mujer
Es una mie

Defining Functions to find the max weight and minimum weight from set of tweets

In [13]:
def max_wt(s1,wt1,ind):
    wt=-1.0
    for s in s1:
        if wt1[s]>wt:
            wt=wt1[s]
            ind=s
    return wt,ind

def min_wt(s1,wt1,ind):
    wt=5000
    for s in s1:
        if wt1[s]<wt:
            wt=wt1[s]
            ind=s
    return wt,ind

Actual DQE algorithm to swap out the tweets which are lesser weighted from final set of expanded queries

In [14]:
delta=1.0

while(delta>0.0):
    Tr2=set()
    for ct in range(0,count_tweets):
        if ct not in Tr:
            Tr2.add(ct)
    delt=1.0
    while(delt>0.0):
        ind1=-1
        min_w,ind1=min_wt(Tr,wt_tweets,ind1)
        ind2=-1
        max_w,ind2=max_wt(Tr2,wt_tweets,ind2)
        if(max_w>min_w):
            Tr.remove(ind1)
            Tr2.add(ind1)
            Tr.add(ind2)
            Tr2.remove(ind2)
        else:
            delt=0.0
    
    wt_temp=Df.dot(adj_mat1.tocsr())
    wt_features=wt_temp.dot(wt_tweets.transpose()) #Updating features
    wt_tweets=adj_mat1.tocsr().transpose().dot(wt_features)+(0.5*adj_mat2.dot(wt_tweets))
    wt_tweets = wt_tweets / wt_tweets.max(axis=0) #Updating tweets
    m,w=max_wt(Tr2,wt_tweets,-1)
    m1,w1=min_wt(Tr,wt_tweets,-1)
    delta=m-m1

Printing Final results

In [15]:
for t in Tr:
    print tweet_dict[tweet_list[t]]

Como es posible que si vas con tu mujer, en un SITP con un solo puesto libre, la hagas sentar en el piso para que tu te sientes en la silla!
Hoy NO es el día de las velitas. Es el día de la Virgen Inmaculada, única mujer que ha concebido sin pasar por el p… https://t.co/aPg3tCeqk7
Usted puede ser la niña mas linda, la mujer mas hermosa, pero si usted es o tiene delirios de ñera, no se me puede hacer atractiva
Ella Mi Reina Mi Gran Mujer La Dueña De Mi Mundo Este Amor Qué No… https://t.co/bFUlAbEald
@CaracolRadio DESPUES DE ESCUCHAR  ESTA ENTREVISTA EN CARACOL RADIO  ,   CORROBORO  QUE @SILVIACORZO  ES UNA MUJER  "EXTRAORDINARIA "
Me gusta la mujer que es frentera y no disimula 🎤
"El reggaeton denigra a la mujer"
- una morronga tercermundista que es moza o le acepta la moza al marido 

Lol la ignorancia😩
En San Cristóbal una mujer entrega en adopción a perros y gatos porque tiene que desalojar su casa. Informes en este teléfono 3102346729.
@AngieMmp buena decisión sabia mujer
Es una mie